In [1]:
from sqlalchemy import create_engine
import os
import time
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load API key for data.gov

with open('data_gov_api.txt', 'r') as file:
    api_key = file.read().split('\n')[0] # Remove new line

conn = create_engine('postgresql://ubuntu@52.53.236.232:5432/collegesc')

data_dir = os.path.join(os.path.curdir, "data")
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

## Helper functions and dictionaries

In [2]:
def find_max_string_length(pd_series):
    """
    For string column in pandas Series, find the maximum length of thes string
    """
    return pd_series.map(lambda x: len(str(x))).max()

def call_collegesc_api(category, api_key, conn, per_page = 50, year = 2013):
    """
    category: the category to be downloaded
    api_key: key for the college sc api
    conn: sql_alchemy engine for querying data definitions
    per_page: number of entries per api call (max: 100)
    year: year to be called (default: 2013)
    calls the college scordcard api given a particular category
    and returns a dataframe transformed from json
    """
    sql_query = """SELECT
                    developername
                FROM 
                    datadefinitions 
                WHERE devcategory ='"""
    
    sql_query += category+"';"
    
    fields = [fields[0] for fields in conn.execute(sql_query).fetchall()]

    
    if(category == "school"):
        join_string = "," + category + "."
        fields_string = "id,location.lat,location.lon" + join_string + join_string.join(fields)
    elif(category == "academics"):
        join_string = "," + category + "."
        fields_string = "id" + join_string + join_string.join(fields)
    else:
        join_string = "," + str(year) + "." + category + "."
        fields_string = "id" + join_string + join_string.join(fields)
    
    college_sc_url = 'https://api.data.gov/ed/collegescorecard/v1/schools.json'
    api_query = {'api_key' : api_key, '_fields' : fields_string, '_per_page' : per_page}
    
    

    req = requests.get(college_sc_url, params=api_query)
    print(req)
    
    pages = req.json()["metadata"]["total"] // per_page
    
    category_df = pd.DataFrame(req.json()["results"])
    
    for page in range(1, pages+1):
        
        api_query["_page"] = page
        i = 0
        while (i < 10):
            try:
                req = requests.get(college_sc_url, params=api_query)
        
                temp_df = pd.DataFrame(req.json()["results"])
        
                category_df = category_df.append(temp_df)
                
                i = 0
                break
            except:
                print("Connection or query error")
                print(req)
                i += 1
                time.sleep(2)
        
        time.sleep(2)
    
    category_df = category_df.reset_index(drop=True)
    
    category_df.columns = category_df.columns.str.replace(category+"\.", "", n = 1)
    category_df.columns = category_df.columns.str.replace(str(year)+"\.", "", n = 1)
        
    
    return category_df
    
def check_highnull_columns(df, threshold=1000):
    """
    takes a dataframe and checks which columns
    have lower than threshold nulls
    returns partial dataframe info with columns with low nulls
    """
    return df.loc[:, df.isnull().sum() < threshold].info()

sql_type_dict = {'integer' : 'integer', 'float' : 'real', 'string' : 'text', 'autocomplete' : 'text'}

def create_datetype_dict(conn, category, type_dict):
    """
    Generates sql query to conn with category to get
    API data categories and convert to Postgres types
    with type dictionary
    Returns type dictionary for that category to generate
    SQL table
    """
    
    datatype_query = """SELECT 
                        developername, datatype
                    FROM
                        datadefinitions
                    WHERE
                        devcategory = '"""
                    
    datatype_query += category+"';"
    
    type_defs = conn.execute(datatype_query).fetchall()
    
    fields = [x[0] for x in type_defs]
    entries = [type_dict[x[1]] for x in type_defs]
    
    sql_type_def_dict = dict(zip(fields, entries))
    sql_type_def_dict['id'] = 'integer'
    
    return sql_type_def_dict
    

## Download College Scorecard Data Dictionary

In [3]:


data_dict_url = "https://collegescorecard.ed.gov/assets/CollegeScorecardDataDictionary.xlsx"

data_dict_file = data_dict_url.split('/')[-1]

data_dict_path = os.path.join(data_dir, data_dict_file)

if not os.path.exists(data_dict_path):
    req = requests.get(data_dict_url)
    
    with open(data_dict_path, 'wb') as file:
        file.write(req.content)


## Load into memory and create table to define variables

In [113]:
sc_data_dict = pd.read_excel(data_dict_path, sheetname='data_dictionary')

In [114]:
sc_data_dict.iloc[300]

NAME OF DATA ELEMENT       Percentage of degrees awarded in Architecture ...
dev-category                                                       academics
developer-friendly name                      program_percentage.architecture
API data type                                                          float
VARIABLE NAME                                                         PCIP04
VALUE                                                                    NaN
LABEL                                      Architecture and Related Services
SOURCE                                                                 IPEDS
NOTES                                        Shown/used on consumer website.
Name: 300, dtype: object

In [115]:
sc_data_dict.iloc[25:28, :]

,NAME OF DATA ELEMENT,dev-category,developer-friendly name,API data type,VARIABLE NAME,VALUE,LABEL,SOURCE,NOTES
25,Control of institution,school,ownership,integer,CONTROL,1.0,Public,IPEDS,Shown/used on consumer website.
26,NaN,school,NaN,NaN,NaN,2.0,Private nonprofit,NaN,NaN
27,NaN,school,NaN,NaN,NaN,3.0,Private for-profit,NaN,NaN


In [116]:
sc_data_dict_nv = sc_data_dict.drop(['VALUE', 'LABEL'], axis = 1)

In [117]:
sc_data_dict_nv.columns = ['Name', 'DevCategory', 'DeveloperName', 'DataType', 'VarName', 'Source', 'Notes']

In [118]:
sc_data_dict_nv = sc_data_dict_nv[sc_data_dict_nv['Name'].notnull()]

In [119]:
sc_data_dict_nv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1734 entries, 0 to 1974
Data columns (total 7 columns):
Name             1734 non-null object
DevCategory      1734 non-null object
DeveloperName    1734 non-null object
DataType         1734 non-null object
VarName          1734 non-null object
Source           1734 non-null object
Notes            274 non-null object
dtypes: object(7)
memory usage: 108.4+ KB


### Find maximum string length in each column so we can define SQL table

In [120]:
string_length_dict = dict()
for column in sc_data_dict_nv.columns:
    string_length_dict[column] = find_max_string_length(sc_data_dict_nv[column])

In [109]:
sc_data_dict_nv.to_csv('datadef.csv', index=False)

In [121]:

query = """CREATE TABLE IF NOT EXISTS datadefinitions (
"""
for column, length in string_length_dict.items():
    query += column + " varchar("+str(length)+") DEFAULT NULL,\n"
    
query += """
PRIMARY KEY (developername)
);"""

In [122]:
query

'CREATE TABLE IF NOT EXISTS datadefinitions (\nName varchar(229) DEFAULT NULL,\nDevCategory varchar(10) DEFAULT NULL,\nDeveloperName varchar(73) DEFAULT NULL,\nDataType varchar(12) DEFAULT NULL,\nVarName varchar(32) DEFAULT NULL,\nSource varchar(11) DEFAULT NULL,\nNotes varchar(2109) DEFAULT NULL,\n\nPRIMARY KEY (developername)\n);'

In [112]:
conn.execute(query)

In [130]:
query2 = """COPY 
                datadefinitions
            FROM 
                '/home/ubuntu/Notebooks/CollegeClassification/datadef.csv'
            DELIMITER ',' CSV HEADER;"""

In [131]:
conn.execute(query2)

In [14]:
query3 = """SELECT * FROM datadefinitions;"""

In [15]:
data_definitions = pd.read_sql_query(query3, conn)

## Use API to gather data for 2013

In [4]:
datatype_query = """SELECT
                        *
                    FROM 
                        datadefinitions
                    WHERE
                        devcategory='root';"""

In [5]:
conn.execute(datatype_query).fetchall()

[('Unit ID for institution', 'root', 'id', 'integer', 'UNITID', 'IPEDS', 'Shown/used on consumer website.'),
 ('8-digit OPE ID for institution', 'root', 'ope8_id', 'integer', 'OPEID', 'IPEDS', 'Shown/used on consumer website.'),
 ('6-digit OPE ID for institution', 'root', 'ope6_id', 'integer', 'OPEID6', 'IPEDS', 'Shown/used on consumer website.'),
 ('Latitude', 'root', 'location.lat', 'float', 'LATITUDE', 'IPEDS', None),
 ('Longitude', 'root', 'location.lon', 'float', 'LONGITUDE', 'IPEDS', None)]

In [217]:
autocomplete_query = """SELECT
                            *
                        FROM
                            datadefinitions
                        WHERE
                            datatype='autocomplete'"""

In [218]:
conn.execute(autocomplete_query).fetchall()

[('Institution name', 'school', 'name', 'autocomplete', 'INSTNM', 'IPEDS', 'Shown/used on consumer website.'),
 ('City', 'school', 'city', 'autocomplete', 'CITY', 'IPEDS', 'Shown/used on consumer website.'),
 ('Institution name aliases', 'school', 'alias', 'autocomplete', 'ALIAS', 'IPEDS', None)]

In [3]:
category_query = """SELECT DISTINCT
                        devcategory
                    FROM
                        datadefinitions;"""

In [4]:
categories = [x[0] for x in conn.execute(category_query).fetchall()]

In [5]:
categories = categories[1:]

In [222]:
datatype_query = """SELECT DISTINCT
                        datatype
                    FROM
                        datadefinitions;"""

In [223]:
conn.execute(datatype_query).fetchall()

[('integer',), ('string',), ('autocomplete',), ('float',)]

#### Remove categories with mostly missing data

In [6]:
categories.remove('academics')
categories.remove('earnings')
categories.remove('completion')
categories.remove('admissions')
categories.remove('cost')
categories

['repayment', 'aid', 'student', 'school']

#### Gather CSV data using API calls

In [10]:
for category in categories:
    print("Starting "+category)
    category_file = os.path.join(data_dir, category + ".csv")
    if not os.path.exists(category_file):
        category_df = call_collegesc_api(category, api_key, conn)
        category_df.to_csv(category_file, index=False)
    print("Completed "+category)

Starting repayment
<Response [200]>
Completed repayment
Starting aid
<Response [200]>
Completed aid
Starting student
Completed student
Starting school
Completed school


## Check resulting CSV files before filling out SQL Database

In [7]:
for category in categories:
    cat_file = os.path.join(data_dir, category+'.csv')
    category_df = pd.read_csv(cat_file)
    print(category)
    print(check_highnull_columns(category_df, threshold=1000))
    print()
    

repayment
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7703 entries, 0 to 7702
Data columns (total 3 columns):
3_yr_default_rate          7488 non-null float64
3_yr_default_rate_denom    7488 non-null float64
id                         7703 non-null int64
dtypes: float64(2), int64(1)
memory usage: 180.6 KB
None

aid
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7703 entries, 0 to 7702
Data columns (total 3 columns):
federal_loan_rate    6806 non-null float64
pell_grant_rate      6806 non-null float64
id                   7703 non-null int64
dtypes: float64(2), int64(1)
memory usage: 180.6 KB
None

student
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7703 entries, 0 to 7702
Data columns (total 29 columns):
FAFSA_applications                                7204 non-null float64
avg_dependent_income.2014dollars                  6835 non-null float64
avg_independent_income.2014dollars                6835 non-null float64
demographics.age_entry                            7204 

Not worth using admissions or cost

In [68]:
sql_table_cat = list(categories)

In [69]:
sql_table_cat.remove('admissions')
sql_table_cat.remove('cost')

In [70]:
sql_table_cat

['repayment', 'aid', 'student', 'school']

#### Create tables (Do not use this, pandas converted intergers with nan to floats)

In [127]:
# for category in sql_table_cat:
#     prim_key = 'id'
#     if (category == 'school'):
#         prim_key += ",zip"

#     cat_type_dict = create_datetype_dict(conn, category, sql_type_dict)
#     cat_create_query = "CREATE TABLE IF NOT EXISTS "+ category +" (\n "

#     for field, var_type in cat_type_dict.items():
#         cat_create_query += '"'+ field + '" ' + var_type + ' DEFAULT NULL,\n '

#     cat_create_query += "PRIMARY KEY ("+prim_key+")\n);"
#     conn.execute(cat_create_query)

#### Fill tables with CSV

In [258]:
for category in sql_table_cat:
    print(category)
    category_path = os.path.join(data_dir, category+'.csv')
    category_df = pd.read_csv(category_path)
    try:
        category_df.to_sql(category, conn, if_exists='fail')
    except:
        print(category+" was already added.")
    

repayment
aid
student
school


#### Verify table contents

In [302]:
example_select = """SELECT * FROM student LIMIT 10;"""

In [303]:
pd.read_sql_query(example_select, conn)

,index,FAFSA_applications,avg_dependent_income.2014dollars,avg_independent_income.2014dollars,demographics.age_entry,demographics.age_entry_squared,demographics.avg_family_income,demographics.avg_family_income_independents,demographics.avg_family_income_independents_log,demographics.avg_family_income_log,...,share_independent_middleincome.30001_48000,share_independent_middleincome.48001_75000,share_independent_students,share_lowincome.0_30000,share_middleincome.30001_48000,share_middleincome.48001_75000,size,students_with_pell_grant,valid_dependency_status,id
0,0,37421.0,45932.0,17369.0,29.0,None,26794.0,17369.0,None,None,...,0.103817,0.046185,0.670025,0.719943,0.126186,0.079608,168.0,0.837150,37421.0,121983
1,1,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,120838
2,2,172178.0,32702.0,23550.0,31.0,None,24342.0,23550.0,None,None,...,0.138113,0.075208,0.913415,0.734078,0.136853,0.077385,479.0,0.874490,172178.0,434973
3,3,496.0,15027.0,9831.0,27.0,None,11643.0,9831.0,None,None,...,NaN,NaN,0.651210,0.927419,NaN,NaN,711.0,NaN,496.0,436012
4,4,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,436021
5,5,197.0,34769.0,12148.0,25.0,None,20301.0,12148.0,None,None,...,NaN,NaN,0.639594,0.761421,0.126904,NaN,39.0,0.923858,197.0,436030
6,6,73.0,39116.0,15608.0,27.0,None,20438.0,15608.0,None,None,...,NaN,NaN,0.794521,0.767123,NaN,NaN,47.0,NaN,73.0,436067
7,7,8413.0,25448.0,10916.0,28.0,None,14239.0,10916.0,None,None,...,0.056557,0.016952,0.771306,0.880780,0.071913,0.027695,624.0,0.945798,8413.0,436191
8,8,4349.0,44674.0,19834.0,27.0,None,28995.0,19834.0,None,None,...,0.115847,0.050273,0.631180,0.694642,0.138423,0.086227,158.0,0.791676,4349.0,436270
9,9,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,164915


In [304]:
student_df = pd.read_csv('./data/student.csv')

In [305]:
student_df.head(10)

,FAFSA_applications,avg_dependent_income.2014dollars,avg_independent_income.2014dollars,demographics.age_entry,demographics.age_entry_squared,demographics.avg_family_income,demographics.avg_family_income_independents,demographics.avg_family_income_independents_log,demographics.avg_family_income_log,demographics.dependent,...,share_independent_middleincome.30001_48000,share_independent_middleincome.48001_75000,share_independent_students,share_lowincome.0_30000,share_middleincome.30001_48000,share_middleincome.48001_75000,size,students_with_pell_grant,valid_dependency_status,id
0,37421.0,45932.0,17369.0,29.0,NaN,26794.0,17369.0,NaN,NaN,0.329975,...,0.103817,0.046185,0.670025,0.719943,0.126186,0.079608,168.0,0.837150,37421.0,121983
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,120838
2,172178.0,32702.0,23550.0,31.0,NaN,24342.0,23550.0,NaN,NaN,0.086585,...,0.138113,0.075208,0.913415,0.734078,0.136853,0.077385,479.0,0.874490,172178.0,434973
3,496.0,15027.0,9831.0,27.0,NaN,11643.0,9831.0,NaN,NaN,0.348790,...,NaN,NaN,0.651210,0.927419,NaN,NaN,711.0,NaN,496.0,436012
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,436021
5,197.0,34769.0,12148.0,25.0,NaN,20301.0,12148.0,NaN,NaN,0.360406,...,NaN,NaN,0.639594,0.761421,0.126904,NaN,39.0,0.923858,197.0,436030
6,73.0,39116.0,15608.0,27.0,NaN,20438.0,15608.0,NaN,NaN,0.205479,...,NaN,NaN,0.794521,0.767123,NaN,NaN,47.0,NaN,73.0,436067
7,8413.0,25448.0,10916.0,28.0,NaN,14239.0,10916.0,NaN,NaN,0.228694,...,0.056557,0.016952,0.771306,0.880780,0.071913,0.027695,624.0,0.945798,8413.0,436191
8,4349.0,44674.0,19834.0,27.0,NaN,28995.0,19834.0,NaN,NaN,0.368820,...,0.115847,0.050273,0.631180,0.694642,0.138423,0.086227,158.0,0.791676,4349.0,436270
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,164915


#### Add primary keys to database

In [283]:
primary_key_add = """ALTER TABLE
                        student
                    ADD PRIMARY KEY
                        (id);"""

In [284]:
conn.execute(primary_key_add)

In [290]:
primary_key_add_zip = """ALTER TABLE
                        school
                    ADD PRIMARY KEY
                        (id,zip);""" #zip used for later additions to database

In [291]:
conn.execute(primary_key_add_zip)

In [293]:
primary_key_add_aid = """ALTER TABLE
                        aid
                    ADD PRIMARY KEY
                        (id);"""

In [294]:
conn.execute(primary_key_add_aid, conn)

In [296]:
primary_key_add_rpmt = """ALTER TABLE
                        repayment
                    ADD PRIMARY KEY
                        (id);"""

In [297]:
conn.execute(primary_key_add_rpmt, conn)

## Example queries

In [300]:
select_example = """SELECT DISTINCT
                        ownership
                    FROM
                        school;"""

In [301]:
conn.execute(select_example).fetchall()

[(2,), (1,), (3,)]

In [312]:
join_example = """SELECT 
                      school.ownership, student."family_income.overall"
                  FROM
                      school
                  JOIN
                      student
                  ON
                      school.id=student.id
                  LIMIT 10;"""

In [313]:
conn.execute(join_example).fetchall()

[(1, 2461.0),
 (1, 5130.0),
 (2, 421.0),
 (1, 2145.0),
 (1, 3642.0),
 (1, 9162.0),
 (1, 1462.0),
 (1, 1789.0),
 (1, 2313.0),
 (1, 4803.0)]

In [320]:
count_example = """SELECT 
                        COUNT(DISTINCT zip)
                   FROM
                       school;"""

In [323]:
conn.execute(count_example).fetchall()[0][0]

5031

In [324]:
count_example2 = """SELECT 
                        COUNT(DISTINCT ownership)
                    FROM
                       school;"""

In [325]:
conn.execute(count_example2).fetchall()[0][0]

3

In [329]:
join3_example = """SELECT
                       school.ownership, 
                       student."demographics.avg_family_income",
                       repayment."3_yr_repayment.male_students_rate"
                   FROM 
                       school 
                   JOIN student ON school.id=student.id
                   JOIN repayment ON student.id=repayment.id
                   WHERE
                       school.ownership=3
                   LIMIT 10;
                        """

In [330]:
conn.execute(join3_example).fetchall()

[(3, 26794.0, 19456.0),
 (3, 24342.0, 236722.0),
 (3, 20301.0, 58.0),
 (3, 20438.0, 56.0),
 (3, 14239.0, 4441.0),
 (3, 28995.0, 2120.0),
 (3, 44533.0, 42.0),
 (3, 9971.0, 0.0),
 (3, 19059.0, 4531.0),
 (3, 49147.0, 30.0)]